In [2]:
import pandas as pd
import numpy as np
import io
import os

In [3]:
def resumen_valores_unicos_limitado(df, max_valores=20):
    columnas = df.columns
    valores_unicos = [df[col].unique()[:max_valores].tolist() for col in columnas]

    df_resumen = pd.DataFrame({
        'columna': columnas,
        'valores_unicos': valores_unicos
    })

    return df_resumen

## Rutas

In [4]:
path='E:/Pedro Casaverde/hackaton/data_2'

## Loading data

### SIAF

In [ ]:
path_siaf_2020 = '/DS03_ORDENES_SIAF/DS03_ORDENESSIAF_2020.csv'
path_siaf_2021 = '/DS03_ORDENES_SIAF/DS03_ORDENESSIAF_2021.csv'
path_siaf_2022 = '/DS03_ORDENES_SIAF/DS03_ORDENESSIAF_2022.csv'
cols_siaf= ["CENT_RUC", "CENT_NOMBRE", "OBANT_NIVEL_GOB", "OBANT_SECTOR", "NOMBRE_DEPARTAMENTO", 
            "NOMBRE_PROVINCIA", "NOMBRE_DISTRITO", "EXPEDIENTE", "NUM_DOC", "FECHA_DOC", "MONTO_NACIONAL_DOC",
            "MONTO_NACIONAL_COMP", "MONTO_NACIONAL_DEV", "MONTO_NACIONAL_GIR", "FLAG_HASTA_8UIT", "GLOSA_COMP",
            "GLOSA_DEV", "GLOSA_GIR", "RUC_PROVEEDOR", "MODALIDAD_COMPRA_DESC", "TIPO_PROCESO_DESC", "COMPROMISO",
            "CLASIFICADOR", "DESCRIPCION_TIPO_TRANSACCION", "DESCRIPCION_GENERICA",   "RUBRO_FINANC", "RUBRO_FINANC_NOMBRE",
            "FUENTE_FINANC", "FUENTE_FINANC_NOMBRE", "DEPARTAMENTO_NOMBRE", "CATEGORIA_PRESUPUESTAL", "GRUPO_GASTO"]


df_siaf_20 = pd.read_csv( path+path_siaf_2020, usecols=cols_siaf)
df_siaf_21 = pd.read_csv( path+path_siaf_2021, usecols=cols_siaf)
df_siaf_22 = pd.read_csv( path+path_siaf_2022, usecols=cols_siaf)

### Proyectos

In [4]:
path_proyectos = '/07_DS_PROYECTOS DE INVERSION/DS07_PROYECTOS_INVERSION.csv'
cols_proyectos= ["COD_SNIP", "NOMBRE_INVERSION", "CENT_RUC","CENT_NOMBRE", "NOMBRE_DEPARTAMENTO", 
            "NOMBRE_PROVINCIA", "NOMBRE_DISTRITO", "MONTO_VIABLE", "COSTO_ACTUALIZADO",
            "ESTADO", "FECHA_VIAB_APROB", "FEC_CIERRE", "MOD_EJECUCION", "MONTO_PIM",
            "MONTO_DEVENGADO", "MONTO_GIRADO", "MONTO_PAGADO", "AVANCE_FISICO_REAL",
            "AVANCE_INVERSION", "EJEC_FIS_PROGRAM_INICIO", "EJEC_FIS_REAL_INICIO", 
            "EJEC_FIS_PROGRAM_FINAL", "EJEC_FIS_REAL_FINAL", "TIPO_INVERSION",
            "DES_FUNCION", "SEC_EJEC", "MONTO_DEV_ACUMULADO", "ULT_PERIODO_DEV", "FASE_INVERSION"]


df_proyectos = pd.read_csv( path+path_proyectos, usecols=cols_proyectos, 
                           dtype={'COD_SNIP':str,'CENT_RUC':str, 'ULT_PERIODO_DEV':str})

C:\Users\Pedro Casaverde\AppData\Local\Temp\ipykernel_26984\1799165673.py:11: DtypeWarning: Columns (22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proyectos = pd.read_csv( path+path_proyectos, usecols=cols_proyectos,


### Proveedores proyectos 1 - 5GB

In [20]:
path_proveedores_1 = '/07_DS_PROYECTOS DE INVERSION/DS07_PROVEEDORES_DE_PROYECTOS.csv'
cols_proveedores_1= ["ANHO", "PROD_PROY", "PROD_PROY_NOMBRE", "SEC_EJEC",
                  "EJECUTORA_NOMBRE", "RUC", "CLASIFICADOR_NOMBRE",
                    "TOTAL_COMPROMISO", "TOTAL_DEVENGADO", "TOTAL_GIRADO"]


df_proveedores_1 = pd.read_csv( path+path_proveedores_1, usecols=cols_proveedores_1,
                               dtype={"RUC":str})

### Proveedores proyectos 2 - 1GB

In [21]:
path_proveedores_2 = '/07_DS_PROYECTOS DE INVERSION/DS07_PROVEEDORES_PROYECTOSINV.csv'
cols_proveedores_2= ["ANHO", "PROD_PROY", "PROD_PROY_NOMBRE", "SEC_EJEC",
                  "EJECUTORA_NOMBRE", "RUC_PROVEEDOR", "CLASIFICADOR_NOMBRE",
                    "TOTAL_COMPROMISO", "TOTAL_DEVENGADO", "TOTAL_GIRADO"]


df_proveedores_2 = pd.read_csv( path+path_proveedores_2,
                               dtype={"RUC_PROVEEDOR":str})

### Banco de inversiones

In [68]:
df_BC_v_1= pd.read_excel(path+'/consulta_avanzada/via_2008.xlsx', 
                         dtype={'Código SNIP':str, 'Código único de inversión':str,
                                'Mes/año primer devengado':str,
                                'Mes/año último devengado':str,
                                'Ubigeo':str},header=5,
                         usecols=['Código SNIP','Monto viable','Código único de inversión',
                                  'Responsable UF','Fecha de registro','Último estudio',
                                  'Fecha de viabilidad','Costo actualizado','Beneficiarios',
                                  'Saldo por financiar','Mes/año primer devengado',
                                  'Mes/año último devengado','Ubigeo'])

In [74]:
# Ruta de la carpeta donde están los archivos Excel
ruta_carpeta = 'E:/Pedro Casaverde/hackaton/data_2/consulta_avanzada'

# Columnas requeridas
columnas_requeridas = ['Código SNIP', 'Monto viable', 'Código único de inversión', 
                       'Responsable UF', 'Fecha de registro', 'Último estudio',
                        'Fecha de viabilidad', 'Costo actualizado', 'Beneficiarios',
                        'Saldo por financiar', 'Mes/año primer devengado', 
                        'Mes/año último devengado', 'Ubigeo']

# DataFrame final donde se concatenarán todos los archivos
df_final = pd.DataFrame()

# Itera sobre cada archivo en la carpeta específica
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.xlsx') and not archivo.startswith('~$'):
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        
        try:
            # Lee el archivo Excel
            df_temp = pd.read_excel(ruta_completa, 
                                    dtype={'Código SNIP': str, 
                                    'Código único de inversión': str, 
                                     'Mes/año primer devengado': str,
                                     'Mes/año último devengado': str,
                                     'Ubigeo': str},
                                     header=5, usecols=columnas_requeridas)
            
            # Agrega una columna con el nombre del archivo
            df_temp['Nombre del Archivo'] = archivo
            
            # Concatena al DataFrame final
            df_final = pd.concat([df_final, df_temp], ignore_index=True)

        except ValueError:
            # Omitir el archivo si no contiene las columnas requeridas
            print(f"El archivo {archivo} fue omitido porque no contiene las columnas requeridas.")



In [79]:
df_final.to_csv(ruta_carpeta+'/consulta_avanzada.csv', index=False)

In [75]:
df_final.head()

,Código único de inversión,Código SNIP,Monto viable,Responsable UF,Fecha de registro,Último estudio,Fecha de viabilidad,Costo actualizado,Beneficiarios,Saldo por financiar,Mes/año primer devengado,Mes/año último devengado,Ubigeo,Nombre del Archivo
0,2488489,2488489,6.625000e+08,LITMAN RAMOS ORTEGA,30/05/2020,NaN,30/05/2020,6.625000e+08,NaN,6.625000e+08,NaN,NaN,150131,aprob_2020.xlsx
1,2488226,2488226,5.452332e+08,LUIS FRANCISCO DEGREGORI CADENILLAS,27/05/2020,NaN,27/05/2020,6.454521e+08,NaN,0.000000e+00,202009,202304,150101,aprob_2020.xlsx
2,2426977,2426977,3.306768e+08,,20/07/2018,NaN,03/09/2018,2.842980e+08,NaN,2.065910e+08,201904,202309,021509,aprob_2020.xlsx
3,2426977,2426977,3.306768e+08,,20/07/2018,NaN,03/09/2018,2.842980e+08,NaN,2.065910e+08,201904,202309,021508,aprob_2020.xlsx
4,2426977,2426977,3.306768e+08,,20/07/2018,NaN,03/09/2018,2.842980e+08,NaN,2.065910e+08,201904,202309,021801,aprob_2020.xlsx


In [78]:
df_final['Nombre del Archivo'].nunique()

8

In [50]:
df_proyectos_01=df_proyectos.drop(columns=["NOMBRE_INVERSION", "CENT_RUC","CENT_NOMBRE", "NOMBRE_DEPARTAMENTO", 
            "NOMBRE_PROVINCIA", "NOMBRE_DISTRITO",])

In [60]:
def generar_df_training(df):
    df_proyectos_01=df.drop(columns=["NOMBRE_INVERSION", "CENT_RUC","CENT_NOMBRE", "NOMBRE_DEPARTAMENTO", 
            "NOMBRE_PROVINCIA", "NOMBRE_DISTRITO",])
    valores_deseados = ['F12B_FASE_SEGUIMIENTO', 'F8_FASE_EJECUCION', 'F7_FASE_FORMULACION']
    df_filtrado = df_proyectos_01[df_proyectos_01['FASE_INVERSION'].isin(valores_deseados)]

    # Agrupar por COD_SNIP y contar valores únicos de FASE_INVERSION
    codigos_con_tres_fases = df_filtrado.groupby('COD_SNIP')['FASE_INVERSION'].nunique()
    codigos_con_tres_fases = codigos_con_tres_fases[codigos_con_tres_fases == 3].index

    # Filtrar el DataFrame original para incluir solo los COD_SNIP seleccionados
    df_resultado = df_proyectos_01[df_proyectos_01['COD_SNIP'].isin(codigos_con_tres_fases)]

    return df_resultado

In [61]:
df_proyectos_01= generar_df_training(df_proyectos)

In [46]:
valores_deseados = ['F12B_FASE_SEGUIMIENTO', 'F8_FASE_EJECUCION', 'F7_FASE_FORMULACION']
df_filtrado = df_proyectos_01[df_proyectos_01['FASE_INVERSION'].isin(valores_deseados)]

# Agrupar por COD_SNIP y contar valores únicos de FASE_INVERSION
codigos_con_tres_fases = df_filtrado.groupby('COD_SNIP')['FASE_INVERSION'].nunique()
codigos_con_tres_fases = codigos_con_tres_fases[codigos_con_tres_fases == 3].index

# Filtrar el DataFrame original para incluir solo los COD_SNIP seleccionados
df_resultado = df_proyectos_01[df_proyectos_01['COD_SNIP'].isin(codigos_con_tres_fases)]

In [48]:
df_resultado.loc[df_resultado.COD_SNIP=='231706'].FASE_INVERSION.value_counts()

FASE_INVERSION
F12B_FASE_SEGUIMIENTO    63
F7_FASE_FORMULACION      14
F8_FASE_EJECUCION        14
Name: count, dtype: int64

In [47]:
df_resultado.head()

,COD_SNIP,MONTO_VIABLE,COSTO_ACTUALIZADO,ESTADO,FECHA_VIAB_APROB,FEC_CIERRE,MOD_EJECUCION,MONTO_PIM,MONTO_DEVENGADO,MONTO_GIRADO,...,EJEC_FIS_PROGRAM_INICIO,EJEC_FIS_REAL_INICIO,EJEC_FIS_PROGRAM_FINAL,EJEC_FIS_REAL_FINAL,TIPO_INVERSION,DES_FUNCION,SEC_EJEC,MONTO_DEV_ACUMULADO,ULT_PERIODO_DEV,FASE_INVERSION
370,231706,5204002,7422365,ACTIVO,11-OCT-17,NaN,ADMINISTRACIÓN DIRECTA,240600.0,234680.19,234680.19,...,01-MAR-18,15-JUL-19,01-JUL-20,28-FEB-23,PROYECTO DE INVERSION,CULTURA Y DEPORTE,1365,6570427.82,202310.0,F12B_FASE_SEGUIMIENTO
371,231706,5204002,7422365,ACTIVO,11-OCT-17,NaN,ADMINISTRACIÓN DIRECTA,240600.0,234680.19,234680.19,...,01-MAR-18,15-JUL-19,01-JUL-20,28-FEB-23,PROYECTO DE INVERSION,CULTURA Y DEPORTE,1365,6570427.82,202310.0,F12B_FASE_SEGUIMIENTO
372,231706,5204002,7422365,ACTIVO,11-OCT-17,NaN,ADMINISTRACIÓN DIRECTA,240600.0,234680.19,234680.19,...,01-MAR-18,15-JUL-19,01-JUL-20,28-FEB-23,PROYECTO DE INVERSION,CULTURA Y DEPORTE,1365,6570427.82,202310.0,F7_FASE_FORMULACION
373,231706,5204002,7422365,ACTIVO,11-OCT-17,NaN,ADMINISTRACIÓN DIRECTA,240600.0,234680.19,234680.19,...,01-MAR-18,15-JUL-19,01-JUL-20,28-FEB-23,PROYECTO DE INVERSION,CULTURA Y DEPORTE,1365,6570427.82,202310.0,F12B_FASE_SEGUIMIENTO
374,231706,5204002,7422365,ACTIVO,11-OCT-17,NaN,ADMINISTRACIÓN DIRECTA,240600.0,234680.19,234680.19,...,01-MAR-18,15-JUL-19,01-JUL-20,28-FEB-23,PROYECTO DE INVERSION,CULTURA Y DEPORTE,1365,6570427.82,202310.0,F12B_FASE_SEGUIMIENTO


In [45]:
df_proyectos_01[['COD_SNIP','FASE_INVERSION']].head()

,COD_SNIP,FASE_INVERSION
0,2383209,F12B_FASE_SEGUIMIENTO
1,2383209,F8_FASE_EJECUCION
2,2383209,F12B_FASE_SEGUIMIENTO
3,2383209,F12B_FASE_SEGUIMIENTO
4,2383209,F12B_FASE_SEGUIMIENTO


In [28]:
df_proyectos.COD_SNIP.nunique()

456727

In [27]:
df_proyectos.FASE_INVERSION.value_counts()

FASE_INVERSION
F12B_FASE_SEGUIMIENTO    1405405
F8_FASE_EJECUCION         602177
F7_FASE_FORMULACION       331418
Name: count, dtype: int64